# KCNH5 Individuals

In [6]:
import pyphetools
from pyphetools.visualization import IndividualTable, QcVisualizer
from IPython.display import display, HTML
from pyphetools.creation import TemplateImporter
template = "input/KCNH5_individuals.xlsx"
hp_json = "../hp.json"
created_by = "0000-0002-0736-9199"
print(f"pyphetools version {pyphetools.__version__}")

pyphetools version 0.9.65


In [9]:
timporter = TemplateImporter(template=template, hp_json=hp_json, created_by=created_by)

In [10]:
individuals, cvalidator = timporter.import_phenopackets_from_template()

HPO version 2024-02-27
Created encoders for 41 fields
 we got OMIM:620537  NM_139318.5
We output 22 GA4GH phenopackets to the directory phenopackets


In [11]:
qc = QcVisualizer(cohort_validator=cvalidator)
display(HTML(qc.to_summary_html()))

Level,Error category,Count
ERROR,CONFLICT,1
WARNING,REDUNDANT,55


In [12]:
table = IndividualTable(cvalidator.get_error_free_individual_list())
display(HTML(table.to_html()))

Individual,Disease,Genotype,Phenotypic features
Proband F (MALE; P13Y),Developmental and epileptic encephalopathy 112 (OMIM:620537),NM_139318.5:c.980G>A (heterozygous),P6M: Bilateral tonic-clonic seizure (HP:0002069)P3Y: Developmental regression (HP:0002376); Hypotonia (HP:0001252)Focal hemiclonic seizure (HP:0006813); Epileptic encephalopathy (HP:0200134); Motor delay (HP:0001270); Delayed speech and language development (HP:0000750); Autistic behavior (HP:0000729); excluded: Bilateral tonic-clonic seizure with focal onset (HP:0007334); excluded: Infantile spasms (HP:0012469); excluded: Myoclonic seizure (HP:0032794); excluded: Generalized non-motor (absence) seizure (HP:0002121); excluded: Status epilepticus (HP:0002133)
patient 25 (MALE; P61Y),Developmental and epileptic encephalopathy 112 (OMIM:620537),NM_139318.5:c.980G>A (heterozygous),Bilateral tonic-clonic seizure (HP:0002069); Epileptic encephalopathy (HP:0200134); Intellectual disability (HP:0001249); Hyperintensity of cerebral white matter on MRI (HP:0030890); Macroglossia (HP:0000158); excluded: Bilateral tonic-clonic seizure with focal onset (HP:0007334); excluded: Focal hemiclonic seizure (HP:0006813); excluded: Infantile spasms (HP:0012469); excluded: Myoclonic seizure (HP:0032794); excluded: Generalized non-motor (absence) seizure (HP:0002121); excluded: Status epilepticus (HP:0002133); excluded: Developmental regression (HP:0002376); excluded: Hypotonia (HP:0001252)
Case 1 (MALE; P4Y),Developmental and epileptic encephalopathy 112 (OMIM:620537),NM_139318.5:c.980G>A (heterozygous),P8M: Bilateral tonic-clonic seizure (HP:0002069)Delayed speech and language development (HP:0000750); Autistic behavior (HP:0000729); excluded: Bilateral tonic-clonic seizure with focal onset (HP:0007334); excluded: Focal hemiclonic seizure (HP:0006813); excluded: Infantile spasms (HP:0012469); excluded: Myoclonic seizure (HP:0032794); excluded: Generalized non-motor (absence) seizure (HP:0002121); excluded: Status epilepticus (HP:0002133); excluded: Motor delay (HP:0001270); excluded: Hyperintensity of cerebral white matter on MRI (HP:0030890)
Case 2 (FEMALE; P4Y),Developmental and epileptic encephalopathy 112 (OMIM:620537),NM_139318.5:c.2020-4A>G (heterozygous),Focal-onset seizure (HP:0007359); Infantile spasms (HP:0012469); Clonic seizure (HP:0020221); excluded: Bilateral tonic-clonic seizure with focal onset (HP:0007334); excluded: Focal hemiclonic seizure (HP:0006813); excluded: Myoclonic seizure (HP:0032794); excluded: Generalized non-motor (absence) seizure (HP:0002121); excluded: Status epilepticus (HP:0002133); excluded: Hyperintensity of cerebral white matter on MRI (HP:0030890)
Case 3 (MALE; P2Y),Developmental and epileptic encephalopathy 112 (OMIM:620537),NM_139318.5:c.962G>A (heterozygous),P7M: Clonic seizure (HP:0020221)excluded: Bilateral tonic-clonic seizure with focal onset (HP:0007334); excluded: Focal hemiclonic seizure (HP:0006813); excluded: Infantile spasms (HP:0012469); excluded: Myoclonic seizure (HP:0032794); excluded: Generalized non-motor (absence) seizure (HP:0002121); excluded: Status epilepticus (HP:0002133); excluded: Motor delay (HP:0001270); excluded: Delayed speech and language development (HP:0000750); excluded: Developmental regression (HP:0002376); excluded: Intellectual disability (HP:0001249); excluded: Hypotonia (HP:0001252); excluded: Hyperintensity of cerebral white matter on MRI (HP:0030890)
Proband 1 (MALE; P4Y),Developmental and epileptic encephalopathy 112 (OMIM:620537),NM_139318.5:c.970A>G (heterozygous),P3M: Focal motor seizure (HP:0011153)Bilateral tonic-clonic seizure with focal onset (HP:0007334); Status epilepticus (HP:0002133); Epileptic encephalopathy (HP:0200134); Global developmental delay (HP:0001263); excluded: Focal impaired awareness motor seizure (HP:0032712); excluded: Focal aware motor seizure (HP:0020217); excluded: Focal tonic seizure (HP:0011167); excluded: Focal hemiclonic seizure (HP:0006813); excluded: Myoclonic seiz

In [13]:
PMID = "PMID:36307226"
df = timporter.create_hpoa_from_phenopackets(pmid=PMID, moi="Autosomal dominant")

[pyphetools] Ingested 22 GA4GH phenopackets.
	Developmental and epileptic encephalopathy 112 (OMIM:620537): n=22
We found a total of 27 unique HPO terms
Extracted disease: Developmental and epileptic encephalopathy 112 (OMIM:620537)
Wrote HPOA disease file to OMIM-620537.tab


In [14]:
hpoa_table_creator.write_data_frame()

Wrote HPOA disease file to OMIM-620537.tab
